# C_b_1 : Recherche de modèles


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import locale
import calendar
import holidays
from datetime import date
from references import *
from src import *

import warnings

warnings.filterwarnings("ignore")
plt.style.use("fivethirtyeight")
pd.options.mode.chained_assignment = None

locale.setlocale(locale.LC_ALL, "fr_CA.UTF-8")

'fr_CA.UTF-8'

In [9]:
df = import_data()
df

,MW,Temp
date,,
2018-12-28 00:00:00,NaN,-10.0
2018-12-28 01:00:00,NaN,-9.0
2018-12-28 02:00:00,NaN,-7.8
2018-12-28 03:00:00,NaN,-6.3
2018-12-28 04:00:00,NaN,-4.1
...,...,...
2022-12-31 19:00:00,23381.99,1.9
2022-12-31 20:00:00,22627.35,2.5
2022-12-31 21:00:00,22004.14,2.1


---

## Prophet

> Prophet is a framework for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust in regard to missing data and shifts in trend, and typically handles outliers well.


### Exemple qui a inspiré ce sujet 

Rob Mulla

https://youtu.be/j0eioK5edqg?si=jW2169K9XA5xdbPP

### Autre exemple avec variables multiples : 

https://github.com/nicolasfauchereau/Auckland_Cycling/blob/master/notebooks/Auckland_cycling_and_weather.ipynb

interpolate so that there are no missing values : utile pour la journée et demi qu'il manque de MW ??

	temp = temp.interpolate(method='linear')



	data_with_regressors = utils.add_regressor(data, temp, varname='temp')

Then we add the extra-regressors to the model using the add_regressor method

	m.add_regressor('temp', prior_scale=0.5, mode='multiplicative')


 